# House Prices - EDA and models comparison

*Autores: David Tejero Ruiz & Miguel Gil Castilla*

El [dataset](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data) ha sido obtenido de la página web de Kaggle en el apartado de competiciones. Este ha sido escogido debido a que está recomendado como un dataset de "juguete" con el que continuar el aprendizaje en python y ciencia del dato a un nivel más o menos básico. Nuestra idea es realizar un exploratory data análisis aplicando técnicas vistas en clase y ampliando estas con algunas ideas captadas de la red, tras esto trataremos nuestro dataset ya modificado para crear un modelo de regresión con el que predecir la variable respuesta *PriceSale*.

Una particularidad de este dataset es su elevado número de atributos (79), lo que lo hace muy adecuado para poner en práctica algunas técnicas vistas de ingeniería de características. Así, podemos extraer información relevante de ellas para realizar una predicción más o menos precisa del precio de venta de las diferentes viviendas.


Dado que en el conjunto test.csv no se proporciona el valor de la variable respuesta (era objetivo obtenerlo para la competición de Kaggle al que pertenece), nos hemos centrado en el conjunto *train.csv*, que hemos renombrado al archivo *data.csv* que se encuentra en este directorio, y será el conjunto de datos que analicemos.

In [ ]:
import numpy as np

# Importamos el conjunto de datos
import pandas as pd
House_prices = pd.read_csv('data.csv')

# Para evitar avisos innecesarios (FutureWarnings):
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)
simplefilter(action='ignore', category=DeprecationWarning)

Notar que Pandas por defecto, trata los valores NA (Not Available) como NaN (Not a Number). Si se analiza la descripción de los datos, NA es un valor que pueden tomar algunos atributos categóricos (no todos), por lo que no se trata de un valor perdido como tal, sino que  indica que no se puede calcular el valor del atributo, porque la vivienda no dispone de el item que se está evaluando. Por ejemplo, en el caso del atributo de calidad de la piscina, NA indica que la vivienda no dispone de piscina, lo cuál es información relevante, y no un valor perdido que tendríamos que imputar.

Hemos analizado la descripción de los diferentes atributos categóricos y aquellos en los que un NA podría aportar información relevante, son:

- Alley

- BsmtQual

- BsmtCond

- BsmtExposure

- BsmtFinType1

- BsmtFinType2

- FireplaceQu

- GarageType

- GarageYrBlt

- GarageFinish

- GarageQual

- GarageCond

- PoolQC

- Fence

- MiscFeature


In [ ]:
na_attributes = ['Alley', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
                 'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageQual', 'GarageCond',
                 'PoolQC', 'Fence', 'MiscFeature']

In [ ]:
# Vemos la cabecera de los datos
House_prices.head()

Como vemos, el dataset está compuesto por 81 columnas, de las cuales 79 serán las características que usaremos en la predicción, y las dos restantes sol el id (que no aporta información), y la variable que queremos predecir (precio de venta de la casa).

Además notamos que existe una riqueza de características alta, encontrando tanto valores numéricos como categóricos, cosa que analizaremos a continuación en el análisis.

In [ ]:
# Vamos a definir un problema de regresión sobre el precio de las casas, por lo que la variable 
# objetivo del dataset será 'SalePrice' (última columna de nuestro dataset como es habitual)
y_data = House_prices['SalePrice']

# Preprocesado de datos

### Análisis de missing values y non-available values

Como comentamos al inicio, el dataset contiene valores NA, y en algunos casos realmente podrían aportar información útil al decir que no se tiene el objeto que tratamos de evaluar en el atributo. Vamos a ver qué atributos contienen valores NA, y cuántos de estos valores NA contienen.

In [ ]:
# Vamos a contar las apariciones de NaNs en nuestro dataset al completo
print("Hay ", House_prices.isnull().sum().sum(), " valores nulos")

In [ ]:
# Separamos el conjunto de datos en atributos con posibles valores NA y 
# atributos con posibles missing values
House_prices_na = House_prices[na_attributes]
House_prices_mv = House_prices.drop(na_attributes, axis=1)

##### Missing values

In [ ]:
# Vemos cuales de los atributos en los que un NaN significa Missing Value 
# tienen realmente valores NaN
mv_counts = House_prices_mv.isnull().sum()
mv_counts = mv_counts[mv_counts > 0]
print("Atributos que contienen missing values: \n{}".format(mv_counts))
print("Hay ", len(mv_counts), " atributos que contienen missing values")


In [ ]:
mv_counts

In [ ]:
# De entre aquellos en los que NA no es un valor posible del atributo,
# y las apariciones de NaN son datos pérdidos, vamos a ver cuáles son 
# numéricos y cuáles no
House_prices_mv_numerics = House_prices_mv[mv_counts.keys()].select_dtypes(include=[np.number])
House_prices_mv_categorics = House_prices_mv[mv_counts.keys()].select_dtypes(exclude=[np.number])
print("Atributos numericos que contienen missing values: \n{}".format(House_prices_mv_numerics.keys().to_list()))
print("Atributos categóricos que contienen missing values: \n{}".format(House_prices_mv_categorics.keys().to_list()))


In [ ]:
# Los atributos numéricos que contienen missing values los vamos a rellenar con la media
# de los valores de la columna
House_prices_mv_numerics = House_prices_mv_numerics.fillna(House_prices_mv_numerics.mean())

# Los atributos categóricos que contienen missing values los vamos a rellenar con el valor
# más frecuente de la columna
House_prices_mv_categorics = House_prices_mv_categorics.fillna(House_prices_mv_categorics.mode().iloc[0])

# Unimos los dos conjuntos de datos
House_prices_mv_fixed = pd.concat([House_prices_mv_numerics, House_prices_mv_categorics], axis=1)

# Guardamos los datos tratados
for key in mv_counts.keys():
    House_prices[key] = House_prices_mv_fixed[key]

# Vamos a comprobar que ya no hay valores nulos en los atributos que hemos tratado
if (House_prices[mv_counts.keys()].isnull().sum().sum() == 0):
    print("No hay valores nulos en estos conjuntos")

##### Non-Available values


Vamos a ver que atributos contienen valores NA entre los que NA aporta información relevante.

In [ ]:
# Vemos cuales de los atributos en los que un NaN significa Missing Value 
# tienen realmente valoress NaN
na_counts = House_prices_na.isnull().sum()
na_counts = na_counts[na_counts > 0]
print("Atributos que contienen Non Available values: \n{}".format(na_counts))
print("Hay ", len(na_counts), " atributos que contienen Non Available values")

In [ ]:
House_prices_na_numerics = House_prices_na.select_dtypes(include=[np.number])
House_prices_na_categorics = House_prices_na.select_dtypes(exclude=[np.number])
print("Atributos numericos que contienen Non Available values: \n{}".format(House_prices_na_numerics.keys().to_list()))
print("Atributos categóricos que contienen Non Available values: \n{}".format(House_prices_na_categorics.keys().to_list()))

Vamos a empezar tratando el caso especial del único atributo numérico que tiene un NA como valor informativo, hablamos del **GarageYrBlt**. Este atributo codifica el año en el que se construyó el garaje de la casa, y un valor NA indica que la casa no tiene garaje. La información de "no tenemos garaje" puede ser relevante, por lo que tenemos que tratar este caso cuidosamente.

Sin embargo, parte de los atributos que analizaremos un poco más adelante: GarageType, GarageFinish, GarageCond y Garageal, son atributos categóricos cuyo valor NA indica igualmente "no tenemos garaje", por lo que esa información en este atributo en particular es redundante.

Por tanto, hemos llegado a la decisión de sustituir los valores NA de GarageYrBlt por la media de los valores no NA de este atributo. De esta forma, cuando posteriormente le apliquemos una normalización estándar al atributo y le restemos la media, obtendremos un valor nulo, y por tanto no afectará a la predicción dado a que así indicaremos que no es relevante ese valor en este atributo.

In [ ]:
House_prices_na_numerics = House_prices_na_numerics.fillna(House_prices_na_numerics.mean())

Para el resto de valores NA, los NaNs indican un posible valor categorico, por lo que vamos a sustituirlos por la cadena "None", para que se traten como un posible valor categórico más.

In [ ]:
# Rellenamos los NaNs con la cadena 'None
House_prices_na_categorics = House_prices_na_categorics.fillna('None')

# Unimos los dos conjuntos de datos tratados
House_prices_na_fixed = pd.concat([House_prices_na_numerics, House_prices_na_categorics], axis=1)

# Guardamos los datos tratados
for key in na_counts.keys():
    House_prices[key] = House_prices_na_fixed[key]

In [ ]:
# Vamos a comprobar que ya no hay valores nulos en nuestro conjunto de datos
if (House_prices.isnull().sum().sum() == 0):
    print("No hay valores nulos en estos conjuntos")

### Fase inicial de ingeniería de características
Vamos a analizar estas sustituticiones con los valores 'None' que repercusión tiene sobre el conjunto de datos. Se mostrará una gráfica con la cantidad de estos valores para valorar si es necesario seguir con la exploración o simplemente eliminar las estancias.

In [ ]:
# Vamos a graficar los valores NA que susituimos por 'None'
import matplotlib.pyplot as plt

ordenados_na = na_counts.sort_values(ascending=False)
plt.bar(ordenados_na.index, ordenados_na)
plt.xticks(rotation='vertical')
plt.ylabel('Número de valores NA')
plt.xlabel('Atributos')
plt.title('Valores NA sustituidos por None')
plt.grid()
plt.show()

# Print de los 4 mayores:
print("Los 4 mayores valores NA sustituidos por None son:")
print(ordenados_na.head(4))

Se observa que existe una cantidad exuberante de datos marcados con None, superando el 50% de las instancias, para 4 de los atributos, concretamente:
 - PoolQC: 1453
 - MiscFeature: 1406 
 - Alley: 1369
 - Fence: 1179
 
Debemos de considerar que el número total de instancias en el dataset es de **1460**.

Para ello dibujaremos el par atributo-variable respuesta para ver la relación entre estos dos, concretamente en los valores que no son None. Con esta comparación veremos si es relevante la información que nos aporta o no el atributo, decidiendo si eliminarlo o no..

In [ ]:
# Pintamos histograma de la variable objetivo, para ver su distribución
plt.hist(y_data, bins=50)
plt.show()

Esto nos da la intuición de dónde se sitúa la media de la variable respuesta y somo es su distribución, con ella podremos decidir si eliminar o no algunso atributos. Nuestra idea es que si los valores que no son 'None' son pocos y se sitúan cercanos a la donde de encuentran la mayoría de instancias (cercanos a la media de la variable respuesta), probablemente estos atributos no aportarán información relevante para la regresión, comparado con la información de otros atributos.

In [ ]:
# Mostramos los valores  de las 4 variables con mayor número de valores NA
# en aquellos que son diferentes de None.

fig, ax = plt.subplots(2, 2, figsize=(15, 7))
ax[0,0].scatter(House_prices['SalePrice'], House_prices[ordenados_na.index[0]])
ax[0,0].set_xlabel('SalePrice')
ax[0,0].set_ylabel(ordenados_na.index[0])
ax[0,0].grid()

ax[0,1].scatter(House_prices['SalePrice'], House_prices[ordenados_na.index[1]])
ax[0,1].set_xlabel('SalePrice')
ax[0,1].set_ylabel(ordenados_na.index[1])
ax[0,1].grid()

ax[1,0].scatter(House_prices['SalePrice'], House_prices[ordenados_na.index[2]])
ax[1,0].set_xlabel('SalePrice')
ax[1,0].set_ylabel(ordenados_na.index[2])
ax[1,0].grid()

ax[1,1].scatter(House_prices['SalePrice'], House_prices[ordenados_na.index[3]])
ax[1,1].set_xlabel('SalePrice')
ax[1,1].set_ylabel(ordenados_na.index[3])
ax[1,1].grid()

plt.show()

Como se puede observar en las gráficas comparativas, se observa que los atributos toman 'None' a lo largo de todo el rango de valores de SalePrice. Vemos que los valores que no son None de los atributos categóricos son pocos y se encuentran además en la zona donde tenemos un alto número de instancias, con lo cual, no aportan información relevante en este problema de regresión. Es por esto por lo que decidimos directamente eliminar estos atributos categóricos.

In [ ]:
# Eliminamos los 4 atributos que más NA contienen 
House_prices = House_prices.drop(ordenados_na.head(4).index, axis=1)

### Construcción de nuestro dataset
Tras haber realizado un primer análisis inicial, vamos a construir el dataset de partida, que será el que usaremos para realizar el análisis exploratorio de datos y la construcción de los modelos de predicción.

In [ ]:
# Definimos el dataset de datos de entrada (X_data) como el dataset sin la variable objetivo
# y sin la variable 'Id' (que no aporta información)
X_data = House_prices.drop(['SalePrice', 'Id'], axis=1)

# Para tratar los diferentes atributos lo primero es distinguir entre las variables cuantitativas 
# (numéricas) y cualitativas (categóricas)
# Ya que el tipo de dato 'object' engloba a las variables categóricas y a las variables de tipo
# # string, se puede usar dicho tipo de dato para distinguir las variables en numéricas y categóricas
numerical_atr = [col for col in X_data.columns if X_data.dtypes[col] != 'object']
categorical_atr = [col for col in X_data.columns if X_data.dtypes[col] == 'object']

print("Hay ", len(numerical_atr), " datos numéricos: ",numerical_atr)
print("Hay ", len(categorical_atr), " datos categóricos: ",categorical_atr)

### Codificación de atributos categóricos

In [ ]:
# Mostramos las variables categoricas
X_data[categorical_atr].head()

Como bien sabemos, estos atributos categóricos debemos transformarlos a valores numéricos para poder trabajar con ellos. Existen diferentes técnicas para realizar esto, y para hacerlo de forma consistente, debemos analizar la naturaleza de los diferentes atributos categóricos.

Notamos que existen una serie de variables categóricas a cuyos atributos podemos asignarles un valor numérico según un orden. Estos casos son cuando hablamos de calidades (a mayor calidad vamos a asignar un valor superior) o cuando cuantificamos con palabras una cantidad (por ejemplo, el LandSlope caracteriza la pendiente del terreno, y podemos asignarle un valor numérico mayor a una mayor pendiente).

In [ ]:
# Basándonos en el archivo data_description.txt codificamos las variables categóricas con información cuantitativa
X_data['LotShape'] = X_data['LotShape'].map({'Reg': 3, 'IR1': 2, 'IR2': 1, 'IR3': 0})
X_data['LandSlope'] = X_data['LandSlope'].map({'Gtl': 0, 'Mod': 1, 'Sev': 2})
X_data['ExterQual'] = X_data['ExterQual'].map({'Ex': 4, 'Gd': 3, 'TA': 2, 'Fa': 1, 'Po': 0})
X_data['ExterCond'] = X_data['ExterCond'].map({'Ex': 4, 'Gd': 3, 'TA': 2, 'Fa': 1, 'Po': 0})
X_data['BsmtQual'] = X_data['BsmtQual'].map({'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1, 'None': 0})
X_data['BsmtCond'] = X_data['BsmtCond'].map({'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1, 'None': 0})
X_data['BsmtExposure'] = X_data['BsmtExposure'].map({'Gd': 4, 'Av': 3, 'Mn': 2, 'No': 1, 'None': 0})
X_data['BsmtFinType1'] = X_data['BsmtFinType1'].map({'GLQ': 6, 'ALQ': 5, 'BLQ': 4, 'Rec': 3, 'LwQ': 2, 'Unf': 1, 'None': 0})
X_data['BsmtFinType2'] = X_data['BsmtFinType2'].map({'GLQ': 6, 'ALQ': 5, 'BLQ': 4, 'Rec': 3, 'LwQ': 2, 'Unf': 1, 'None': 0})
X_data['HeatingQC'] = X_data['HeatingQC'].map({'Ex': 4, 'Gd': 3, 'TA': 2, 'Fa': 1, 'Po': 0})
X_data['KitchenQual'] = X_data['KitchenQual'].map({'Ex': 4, 'Gd': 3, 'TA': 2, 'Fa': 1, 'Po': 0})
X_data['Functional'] = X_data['Functional'].map({'Typ': 7, 'Min1': 6, 'Min2': 5, 'Mod': 4, 'Maj1': 3, 'Maj2': 2, 'Sev': 1, 'Sal': 0})
X_data['FireplaceQu'] = X_data['FireplaceQu'].map({'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1, 'None': 0})
X_data['GarageFinish'] = X_data['GarageFinish'].map({'Fin': 3, 'RFn': 2, 'Unf': 1, 'None': 0})
X_data['GarageQual'] = X_data['GarageQual'].map({'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1, 'None': 0})
X_data['GarageCond'] = X_data['GarageCond'].map({'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1, 'None': 0})
X_data['PavedDrive'] = X_data['PavedDrive'].map({'Y': 2, 'P': 1, 'N': 0})

In [ ]:
# Estas dos características podemos tratarlas como binarias al tener solo dos posibles valores
X_data['CentralAir'] = X_data['CentralAir'].map({'Y': 1, 'N': 0})
X_data['Street'] = X_data['Street'].map({'Pave': 1, 'Grvl': 0})

Las que nos quedan, son variables categóricas que no podemos ordenar, y por tanto, debemos aplicar una codificación one-hot. Para ello, vamos a usar la función OneHotEncoder de sklearn, que nos permite realizar esta codificación de forma sencilla.

In [ ]:
# Recalculamos los valores que siguen siendo categóricos
categorical_atr = [col for col in X_data.columns if X_data.dtypes[col] == 'object']
print("Quedan ", len(categorical_atr), " datos categóricos: ",categorical_atr)

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# Usamos OneHotEncoder para codificar las variables categóricas
encoder = OneHotEncoder(sparse=False)
X_data_encoded = pd.DataFrame(encoder.fit_transform(X_data[categorical_atr]))
X_data_encoded.columns = encoder.get_feature_names(categorical_atr)

# Borramos las variables categóricas originales y añadimos las nuevas codificadas
X_data = X_data.drop(categorical_atr ,axis=1)
X_data = pd.concat([X_data, X_data_encoded], axis=1)

# Mostramos los datos de entrada
X_data.head()

Mencionar aunque 214 parezcan muchos atributos, gracias al label encoding manual que hemos realizado arriba usando información de la descripción de los datos, nos hemos ahorrado hacer one-hot-encoding de todos los atributos categóricos, lo cual supondría obtener bastantes más atributos, aumentando consideralmente la complejidad del problema.

Notar que hemos optado por usar one-hot-encoding para las variables en las que no podemos definir un orden, y no otro tipo de codificaciones, para no introducir un orden artificial que no existiría en la realidad.

In [ ]:
# Probamos que no hay valores nulos en el conjunto final de datos
if (X_data.isnull().sum().sum() == 0):
    print("No hay valores nulos en estos conjuntos")

# Probamos que no quedan variables categóricas
categorical_atr = [col for col in X_data.columns if X_data.dtypes[col] == 'object']
print("Quedan ", len(categorical_atr), " datos categóricos: ",categorical_atr)

### Ingeniería de características avanzada

##### Multicolinealidad

In [ ]:
# Obtenermos la matriz de correlación de los datos de entrada
correlation_matrix = X_data.corr()

In [ ]:
# Coeficiente de correlación de Pearson entre los atributos
import seaborn as sns
plt.figure(figsize=(15,15))
plt.title('Coeficiente de Correlación de Pearson entre los atributos', y=1.05, size=15)
sns.heatmap(correlation_matrix, cmap='viridis',vmax=1)
plt.show()

In [ ]:
# Vamos a identificar pares de atributos con correlación mayor a 0.7
# y vamos a eliminar uno de los dos atributos de cada par
# para evitar la multicolinealidad

# Mantenemos una lista de atributos eliminados para no eliminarlos dos veces, y simplifcamos

# La matriz es simétrica, por lo que definimos los bucles para recorrer solo la mitad superior
# de la matriz

# Tenemos en cuenta tanto correlación positiva como negativa, por lo que usamos el valor absoluto

umbral = 0.7
atributos_eliminados = []
for i in range(len(correlation_matrix.columns)): 
    for j in range(i+1, len(correlation_matrix.columns)): 
        if (np.abs(correlation_matrix.iloc[i, j]) >= umbral):
            if (correlation_matrix.columns[j] not in atributos_eliminados):
                atributo = correlation_matrix.columns[j]
                atributos_eliminados.append(atributo)
                print("Eliminamos el atributo ", atributo, " por tener una correlación mayor a ", umbral, " con el atributo ", correlation_matrix.columns[i])

print("Se van a eliminar ", len(atributos_eliminados), " atributos")

In [ ]:
# Eliminamos los atributos de la lista de atributos a eliminar
X_data = X_data.drop(atributos_eliminados, axis=1) 

##### Metodos de filtrado para selección de características
A continuación, vamos a estudiar la correlación de los atributos con la variable respuesta, para ver si podemos eliminar algunos atributos que no aporten información relevante para la predicción.

In [ ]:
correlation_matrix = pd.DataFrame(pd.concat([X_data, y_data], axis=1)).corr()
correlation_matrix = correlation_matrix['SalePrice']

# Ploteamos la matriz de correlación de los datos de entrada
plt.figure(figsize=(15, 0.5))
plt.title('Coeficiente de Correlación de Pearson entre los atributos', y=1.05, size=15)
sns.heatmap(correlation_matrix.to_frame().T, cmap='viridis', vmax=1, vmin=-1)
plt.xticks(rotation=90)
plt.show()

In [ ]:
# Vamos a eliminar los atributos que tengan una correlación pequeña con la variable objetivo
umbral = 0.025
atributos_a_eliminar = []
for i in range(len(correlation_matrix)):
    if (np.abs(correlation_matrix[i]) < umbral):
        atributo = correlation_matrix.index[i]
        atributos_a_eliminar.append(atributo)
        print("Eliminamos el atributo ", atributo, " por tener una correlación menor a ", umbral, " con la variable objetivo")

print("Se van a eliminar ", len(atributos_a_eliminar), " atributos")

In [ ]:
# Eliminamos los atributos de la lista de atributos a eliminar
X_data = X_data.drop(atributos_a_eliminar, axis=1)

In [ ]:
X_data.head()

### División en entrenamiento, validación y prueba

In [ ]:
# Dividimos el conjunto de datos en tres subconjuntos: entrenamiento, validación y test
# En una proprorción 50% - 20% - 30% respectivamente

train_size = 0.89
val_size = 0.01
test_size = 0.1

from sklearn.model_selection import train_test_split
X_train, X_aux, y_train, y_aux = train_test_split(X_data, y_data, test_size=(1.0-train_size), random_state=41)
X_val, X_test, y_val, y_test = train_test_split(X_aux, y_aux, test_size=test_size/(val_size+test_size), random_state=41)

# Mostramos los tamaños de los conjuntos de datos
print("Tamaño del conjunto de entrenamiento: ", len(X_train))
print("Tamaño del conjunto de validación: ", len(X_val))
print("Tamaño del conjunto de test: ", len(X_test))

### Normalización de atributos

In [ ]:
# Normalizamos las variables numéricas para que todas tengan media 0 y desviación típica 1
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

X_train = pd.DataFrame(X_train, columns=X_data.columns)
X_test = pd.DataFrame(X_test, columns=X_data.columns)
X_val = pd.DataFrame(X_val, columns=X_data.columns)

# Mostramos los datos de entrada
X_train.head()

X_data = pd.concat([X_train, X_val, X_test])



# Modelos

In [ ]:
# # Entrenamos gradient boosting con los datos de entrenamiento
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error

# Entrenamos el modelo con los datos de entrenamiento
gbr = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05, max_depth=4, max_features='sqrt', min_samples_leaf=15, min_samples_split=10, loss='huber', random_state =42)                             
gbr.fit(X_train, y_train)
y_pred = gbr.predict(X_test)
err = mean_absolute_error(y_test, y_pred)/np.mean(y_test)
print ('%.2f'%err)
y_pred = gbr.predict(X_train)
err = mean_absolute_error(y_train, y_pred)/np.mean(y_train)
print ('%.2f'%err)

In [ ]:
# See sales
plt.scatter(y_train, y_pred)
plt.grid()

COSAS QUE HAY QUE HACER:

**Preprocesado de datos**
- Eliminar columnas con muchos valores perdidos
- One-hot encoding / Label encoding (categorical variables)
- Normalizar datos: StandardScaler, MinMaxScaler, RobustScaler 
- Eliminar outliers: IQR, Z-score, etc.  (?)
- Discretizado de variables continuas (?): Binning, etc.


**Ingeneering features**
- Reducción de dimensionalidad: *PCA*, LDA, etc.
- Selección de variables - Filter Methods: Correlation, Chi2, ANOVA, etc.
- Multicolinealidad: VIF, etc. 
- Crear nuevas variables.

**Posibles Modelos**
- Regresión lineal
- Regresión polinómica
- Regresión Ridge
- Regresión Lasso
- Regresión ElasticNet
- SVR
- Random Forest
- **Gradient Boosting**
- XGBoost
- LightGBM
- CatBoost

**Evaluación de modelos**
- MSE
- k-fold cross validation
- Recall, Precision, F1-score, etc. (?)